In [1]:
from sklearn.datasets import make_classification

In [2]:
dataset = make_classification(n_samples=10_000, n_features=20, n_classes=2)
x, y = dataset

In [3]:
x, y

(array([[ 0.16276476,  0.41671867, -0.88394142, ..., -0.31492062,
         -0.05024597, -0.51725234],
        [-1.24710286, -0.73957774,  2.40476732, ...,  1.11348456,
          0.42394959,  1.58352886],
        [ 0.28086439,  1.35074467,  0.52655531, ...,  1.62105396,
          0.34334042,  1.7146822 ],
        ...,
        [ 0.49054593, -0.78052256,  0.40249254, ..., -0.579728  ,
         -0.15259969,  0.29218807],
        [-1.52102168,  0.1494984 , -0.59845799, ..., -0.28498895,
         -0.75400824, -0.37005728],
        [-0.93142201,  0.24564272, -0.90727319, ..., -1.05812923,
          1.09671567, -0.4668389 ]]),
 array([1, 1, 1, ..., 1, 1, 0]))

In [4]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

sx = ss.fit_transform(x)

In [18]:
import tensorflow as tf

p = .1

In [6]:
inputs = tf.keras.layers.Input((20,), name='numeric_inputs')

In [7]:
x = tf.keras.layers.Dropout(p)(inputs)
x = tf.keras.layers.Dense(100, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(10, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)

In [20]:
model = tf.keras.models.Model(inputs=inputs, outputs=x)
model.compile(optimizer='rmsprop', 
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [21]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 numeric_inputs (InputLayer)  [(None, 20)]             0         
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 dense (Dense)               (None, 100)               2100      
                                                                 
 batch_normalization (BatchN  (None, 100)              400       
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 20)                2020      
                                                           

In [22]:
import numpy as np

def bootstrap_sample_generator(batch_size):
    while (True):
        batch_idx = np.random.choice(
            sx.shape[0], batch_size
        )
        yield ({'numeric_inputs': sx[batch_idx]},
              {'output': y[batch_idx]})

In [23]:
batch_size = 32

model.fit_generator(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=10_000 // batch_size,
    epochs=5,
    max_queue_size=10
)

Epoch 1/5


C:\Users\serge\AppData\Local\Temp\ipykernel_31200\1425163714.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


312/312 [==============================] - 2s 2ms/step - loss: 0.4441 - accuracy: 0.7907
Epoch 2/5
312/312 [==============================] - 1s 2ms/step - loss: 0.3122 - accuracy: 0.8776
Epoch 3/5
312/312 [==============================] - 1s 2ms/step - loss: 0.3038 - accuracy: 0.8778
Epoch 4/5
312/312 [==============================] - 1s 2ms/step - loss: 0.2894 - accuracy: 0.8898
Epoch 5/5
312/312 [==============================] - 1s 2ms/step - loss: 0.2802 - accuracy: 0.8917
